Setting the environment:

In [ ]:
from bertopic import BERTopic
import pandas as pd
import re
import numpy
from os.path import join
from umap import UMAP
from collections import Counter
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"
from sklearn.feature_extraction.text import CountVectorizer

Importing the dataset:

In [ ]:
src = "../../data/tweets"
fname = "combined_US_politician_twitter_timelines_2010-11-06_to_2022-03-16_lemmatized_text.csv.gzip"
data = pd.read_csv(join(src, fname), encoding="utf-8").dropna()
data.head(2)

Cleaning the data and creating a list of docs for the model:

In [4]:
data.text = data.apply(lambda row: re.sub(r"http\S+", "", str(row.text)).lower(), 1)
docs = data.text.to_list()

Fitting the model (increase `n_neigbors` and/or `min_topic_size` if too many topics):

In [5]:
umap_model = UMAP(n_neighbors=100, n_components=5, metric='cosine', low_memory=False)
vectorizer_model = CountVectorizer(min_df=50)
topic_model = BERTopic(verbose=True, nr_topics="auto", min_topic_size=200, umap_model=umap_model, top_n_words=20, vectorizer_model=vectorizer_model)
topics, probs = topic_model.fit_transform(docs)

Batches: 100%|██████████| 48206/48206 [3:12:49<00:00,  4.17it/s]  
2022-05-05 12:37:20,186 - BERTopic - Transformed documents to Embeddings
2022-05-05 21:51:38,008 - BERTopic - Reduced dimensionality with UMAP
2022-05-05 22:02:19,430 - BERTopic - Clustered UMAP embeddings with HDBSCAN
2022-05-05 22:06:19,405 - BERTopic - Reduced number of topics from 485 to 378


Saving and loading the model:

In [ ]:
topic_model.save("twitter_lemmatized")

In [ ]:
topic_model = BERTopic.load("twitter_lemmatized") #insert path

Checking how many topics the model found:

In [6]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,867530,-1_to_and_in_be
1,0,22275,0_obamacare_aca_care_condition
2,1,18886,1_voting_vote_ballot_election
3,2,14776,2_abortion_woman_right_life
4,3,12707,3_gun_violence_background_shooting
...,...,...,...
373,372,206,372_heart_disease_february_awareness
374,373,206,373_hand_produce_donate_covid19
375,374,205,374_rush_radio_conservative_icon
376,375,204,375_conflict_condemn_civilian_violation


Visualising the topics (call `x.write_html(path)` to save the graphs):

In [7]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_heatmap(n_clusters=20)

Dynamic Topic Modelling (topics over time):

In [ ]:
topics, probs = topic_model.transform(docs)

In [ ]:
timestamps = data.created_at.to_list()
topics_over_time = topic_model.topics_over_time(docs, topics, timestamps, nr_bins=20)

Creating topics per class dataframe to check for topic distribution over party/components

In [ ]:
topics_per_class = topic_model.topics_per_class(docs, topics=topics, classes=data["classes"], global_tuning=True)

In [9]:
topic_model.visualize_topics_per_class(topics_per_class, top_n_topics=100)

In [10]:
dst = "../../data/tweets"
fname = "topics_per_class.csv"
topics_per_class.to_csv(join(dst, fname), index=False)

Preparing data for wordclouds (use with specific topics)

In [89]:
topics_ids = [] #insert topic id(s)
topics_words = []
topic_embeddings = []

In [90]:
for id in topics_ids:
    word_list = []
    embedding_list = []
    for i in range(0,20): # insert how many words you want to display (=< top_n_words in model computation)
        word = topic_model.get_topic(id)[i][0]
        word_list.append(word)
        embedding = topic_model.get_topic(id)[i][1]
        embedding_list.append(embedding)
    topics_words.append(word_list)
    topic_embeddings.append(embedding_list)

In [91]:
wordcloud_df = pd.DataFrame()
wordcloud_df["topic_ids"] = topics_ids
wordcloud_df["topic_words"] = topics_words
wordcloud_df["topic_embeddings"] = topic_embeddings

In [92]:
dst = "../../data/tweets"
fname = "key_topics.csv"
wordcloud_df\
    .set_index(["topic_ids"])\
    .apply(pd.Series.explode)\
    .reset_index()\
    .to_csv(join(dst, fname), index=False)